In [1]:
# If first time running this notebook restart after running this cell. Otherwise packages will not be avalible in the kernal.
! pip install pointpats pandas numpy shapely folium ipywidgets

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import numpy as np
import folium
from ipywidgets import interact, interact_manual, IntText
import pandas as pd
from shapely import intersection
from shapely.geometry import MultiPoint, Polygon, Point
from shapely import voronoi_polygons

from voronoi import weightedVoroniDigram

def random_color():
    color = np.random.randint(16, 256, size=3)
    color = [str(hex(i))[2:] for i in color]
    return '#' + ''.join(color).upper()

def create_point(row):
    return Point(row['LATITUDE'], row['LONGITUDE'])

In [3]:
data = pd.read_csv("bike_share_data/cleaned_data.csv")
data['point'] = data.apply(create_point, axis=1)

### Voronoi digram plotter

In [4]:
def plot_voronoi(df):
    '''method to create GUI'''
    points = df[['LATITUDE','LONGITUDE']].to_numpy()
    print(f"Total number of stations: {len(points)}")
    map_center = points.mean(axis=0)
    m = folium.Map(location=map_center, zoom_start=10)

    for index, row in df.iterrows():
            folium.Polygon(
                        locations=row['poly'],
                        color="blue",
                        weight=4,
                        fill_color=random_color(),
                        fill_opacity=0.5,
                        fill=True,
                        popup="Voronoi region",
                        ).add_to(m)
            
            folium.CircleMarker(location=[row['LATITUDE'],row['LONGITUDE']],
                            radius=2,
                            weight=2).add_to(m)
    
    # Display map
    display(m)
my_interact_manual = interact_manual.options(manual_name="Submit")
a = IntText(description="Add stations:", value=4, min=0, max=1000)
r = IntText(description="Remove:", value=2, min=0, max=700)

@my_interact_manual(a=a, r=r)
def adjust_points(a, r):
    boundary = Polygon([(38.7, -76.8), (38.7,-77.4), (39.2,-77.4), (39.2,-76.8)])
    points = weightedVoroniDigram.find_optimal_voronoi_digram(a, r, data.copy(), boundary)
    weightedVoroniDigram.evaluate(data.copy(), points.copy(), boundary)
    plot_voronoi(points)

interactive(children=(IntText(value=4, description='Add stations:'), IntText(value=2, description='Remove:'), …